<a href="https://colab.research.google.com/github/nueveonce/Etapa3DS/blob/main/Etapa3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto NLP: Análisis de Ofertas de Trabajo en LinkedIn

# Introduccion

## 1. Descripción del Problema de Negocio

Las empresas publican miles de ofertas de trabajo diariamente en plataformas como LinkedIn. Analizar manualmente estas descripciones para extraer información relevante (habilidades requeridas, tipo de contrato, sentimiento general de la oferta, etc.) es una tarea ardua y consume mucho tiempo. Existe la necesidad de automatizar este análisis para identificar tendencias del mercado laboral, optimizar la redacción de ofertas y facilitar a los candidatos la búsqueda de puestos adecuados.


# 2. Objetivo General
Aplicar técnicas de Procesamiento de Lenguaje Natural (NLP) para analizar y extraer información valiosa de las descripciones de puestos de trabajo publicados en LinkedIn. Específicamente, se buscará:
 * Preprocesar el texto de las descripciones.
 * Identificar las palabras y términos más frecuentes y significativos.
 * Analizar el sentimiento general expresado en las descripciones.
 * (Opcional Avanzado) Desarrollar un modelo de clasificación para predecir alguna característica de la oferta (ej. "seniority" inferida).

## 3. Origen de los Datos
El dataset proviene de LinkedIn y contiene información sobre ofertas de trabajo. La columna principal para el análisis NLP será 'description'.

In [28]:
# importacion de librerias
import pandas as pd
import re
import string
import nltk
# Carga de datos
df = pd.read_csv('clean_jobs.csv')

# 4.Definición de las Variables:

id: Identificador único del trabajo (Numérico).
* title: Nombre del puesto de trabajo (Texto). Potencialmente útil para NLP, podría combinarse con 'description' o usarse para filtrar.
* company: Nombre de la empresa (Texto).
* location: Nombre del país del puesto de trabajo (Texto).
* link: URL de LinkedIn (Texto).
source: Siempre es LinkedIn (Texto).
* date_posted: Fecha de publicación del puesto de trabajo (Fecha/Texto). Podría usarse para análisis de tendencias temporales, pero no directamente para NLP del contenido.
* work_type: (NaN) - No disponible/Vacío.
* employment_type: (NaN) - No disponible/Vacío.
* description: Descripción del trabajo (Texto). Esta será la columna principal para el análisis NLP.

In [29]:
df.head(5)

,id,title,company,location,link,source,date_posted,work_type,employment_type,description
0,1,Data Analyst,Meta,"New York, NY",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,NaN,NaN,The Social Measurement team is a growing team ...
1,2,Data Analyst,Meta,"San Francisco, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,NaN,NaN,The Social Measurement team is a growing team ...
2,3,Data Analyst,Meta,"Los Angeles, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,NaN,NaN,The Social Measurement team is a growing team ...
3,4,Data Analyst,Meta,"Washington, DC",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,NaN,NaN,The Social Measurement team is a growing team ...
4,5,Data Analyst II,Pinterest,"Chicago, IL",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-16,NaN,NaN,About Pinterest\n\nMillions of people around t...


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               327 non-null    int64  
 1   title            327 non-null    object 
 2   company          327 non-null    object 
 3   location         327 non-null    object 
 4   link             327 non-null    object 
 5   source           327 non-null    object 
 6   date_posted      327 non-null    object 
 7   work_type        0 non-null      float64
 8   employment_type  0 non-null      float64
 9   description      327 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 25.7+ KB


# Busqueda de valores nulos

In [31]:
df.isnull().sum()

,0
id,0
title,0
company,0
location,0
link,0
source,0
date_posted,0
work_type,327
employment_type,327
description,0


## Eliminacion de velores nulos

In [32]:
# Se decide eliminar las colu,nas work_type y employment_type ya que no contienen valores en alguna fila
df = df.drop(['work_type', 'employment_type'], axis=1)

# Quitar simbolos y signos de puntuación

In [33]:


def limpiar_texto(texto):
    # Convertir a minúsculas (se hará después, pero puede hacerse aquí también)
    # texto = texto.lower()
    # Quitar URLs
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    # Quitar emails
    texto = re.sub(r'\S*@\S*\s?', '', texto)
    # Quitar saltos de línea y caracteres especiales (manteniendo espacios)
    texto = re.sub(r'\s+', ' ', texto) # Reemplazar múltiples espacios con uno solo
    texto = re.sub(r'(\r\n|\r|\n)', ' ', texto) # Reemplazar saltos de línea con espacio
    # Quitar caracteres que no sean alfanuméricos (excepto espacios)
    # texto = re.sub(r'[^\w\s]', '', texto) # Esto quitaría también acentos
    # Una mejor aproximación es quitar solo la puntuación explícitamente
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    # Quitar números si no son relevantes (opcional)
    # texto = re.sub(r'\d+', '', texto)
    return texto.strip() # Eliminar espacios al inicio y al final

df['descripcion_limpia'] = df['description'].astype(str).apply(limpiar_texto)
print("\nEjemplo de descripción limpia:")
print(df['descripcion_limpia'].iloc[0])


Ejemplo de descripción limpia:
The Social Measurement team is a growing team with highvisibility within the Communications organization that is being tasked with measuring the efficacy and impact of our socialfirst Communications and Marketing efforts Youll play a unique role in shaping the way we approach reporting define success and leverage our own data to support the Communications team In this role you will employ data mining techniques to evaluate the success and efficacy of our socialfirst Communications and Marketing strategies This role is also focused on social listening where youll be responsible for analyzing social conversation around specific topic areas The ideal candidate will possess not only the technical proficiency to devise analytical solutions but also have experience with datastorytelling Data Analyst Responsibilities Employ Natural language processing NLP and social listening analytical techniques to understand how people discuss specific topics Create data pip

In [34]:
# mostamos solo las columnas descripcion_limpia y description
df[['descripcion_limpia', 'description']].head(5)

,descripcion_limpia,description
0,The Social Measurement team is a growing team ...,The Social Measurement team is a growing team ...
1,The Social Measurement team is a growing team ...,The Social Measurement team is a growing team ...
2,The Social Measurement team is a growing team ...,The Social Measurement team is a growing team ...
3,The Social Measurement team is a growing team ...,The Social Measurement team is a growing team ...
4,About Pinterest Millions of people around the ...,About Pinterest\n\nMillions of people around t...


# Tokenizar

In [35]:
# Descargar recursos necesarios de NLTK (solo la primera vez)
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

def tokenizar_texto(texto):
    tokens = nltk.word_tokenize(texto, language='spanish' if any(c in 'áéíóúñ' for c in texto.lower()) else 'english')
    # Considerar el idioma de tus descripciones. Si son mayormente en inglés:
    # tokens = nltk.word_tokenize(texto, language='english')
    return tokens

df['tokens'] = df['descripcion_limpia'].apply(tokenizar_texto)
print("\nEjemplo de tokens:")
print(df['tokens'].iloc[0][:20]) # Mostrar los primeros 20 tokens

AttributeError: module 'nltk.downloader' has no attribute 'DownloadError'